# Procesamiento y Análisis de Datos LiDAR

Este notebook realiza un análisis completo de datos LiDAR incluyendo:
- Limpieza y filtrado de outliers
- Clasificación de ground y non-ground
- Clasificación de coberturas (suelo, vegetación, edificios)
- Generación de modelos digitales (DTM, DSM, nDSM)
- Segmentación y métricas de edificios
- Exportación de resultados en formato GeoTIFF

## 1. Importar Librerías Necesarias

Se importan las siguientes herramientas:
- **laspy**: Lectura de archivos LAS
- **numpy, pandas**: Procesamiento de datos numéricos
- **sklearn**: Algoritmos de machine learning (vecinos cercanos, clustering)
- **scipy**: Interpolación espacial y árboles espaciales
- **rasterio**: Lectura/escritura de archivos geoespaciales
- **matplotlib**: Visualización de datos

In [ ]:
import os
import glob

import numpy as np
import pandas as pd

import laspy
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import rasterio
from rasterio.transform import from_origin

from sklearn.cluster import DBSCAN

from scipy.interpolate import griddata

from scipy.spatial import cKDTree

## 2. Localizar y Listar Archivos LAS

Se buscan todos los archivos con extensión `.las` en el directorio `data`.
Estos archivos contienen las nubes de puntos LiDAR a procesar.

In [ ]:
DATA_DIR = "data"
las_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.las")))

print(f"Archivos encontrados: {len(las_files)}")
for f in las_files:
    print(" -", os.path.basename(f))

## 3. Función: Limpieza de Outliers LiDAR

Esta función elimina puntos anómalos usando dos métodos:
1. **Filtro global por Z**: Elimina puntos con elevaciones muy alejadas de la media (±3 desv. estándar)
2. **Filtro estadístico por vecindad (kNN)**: Usa distancias a vecinos cercanos para detectar outliers aislados

Parámetros:
- `k`: Número de vecinos a considerar (default: 8)
- `z_thresh`: Umbral en desviaciones estándar (default: 3.0)
- `knn_mult`: Multiplicador para el umbral de distancia (default: 2.5)

In [ ]:
def clean_lidar_points(x, y, z, k=8, z_thresh=3.0, knn_mult=2.5):
    """
    Limpieza de outliers LiDAR:
    1. Filtro global por Z
    2. Filtro estadístico por vecindad (kNN)
    """

    n_total = len(z)

    # --- Filtro Z global ---
    z_mean = np.mean(z)
    z_std = np.std(z)

    z_min = z_mean - z_thresh * z_std
    z_max = z_mean + z_thresh * z_std

    mask_z = (z >= z_min) & (z <= z_max)

    # --- Filtro kNN ---
    coords = np.column_stack((x[mask_z], y[mask_z], z[mask_z]))

    nbrs = NearestNeighbors(n_neighbors=k + 1).fit(coords)
    distances, _ = nbrs.kneighbors(coords)

    mean_dist = distances[:, 1:].mean(axis=1)
    threshold = mean_dist.mean() + knn_mult * mean_dist.std()

    mask_knn = mean_dist <= threshold

    # --- Máscara final ---
    mask_clean = np.zeros(n_total, dtype=bool)
    mask_clean[np.where(mask_z)[0][mask_knn]] = True

    stats = {
        "total_points": n_total,
        "clean_points": int(mask_clean.sum()),
        "removed_points": int(n_total - mask_clean.sum())
    }

    return mask_clean, stats

## 4. Función: Clasificación Ground/Non-Ground

Clasifica puntos como suelo o no-suelo usando una grilla XY:
- Divide el área en celdas de tamaño `cell_size`
- En cada celda, identifica el terreno usando el percentil 5 de elevaciones
- Un punto se considera suelo si su Z está cerca del terreno local

Parámetros:
- `cell_size`: Tamaño de celdas de grilla (default: 2.0 m)
- `z_threshold`: Margen vertical para clasificar como suelo (default: 0.5 m)
- `ground_percentile`: Percentil para estimar altura del terreno (default: 5)

In [ ]:
def classify_ground(x, y, z, cell_size=2.0, z_threshold=0.5, ground_percentile=5):
    """
    Clasificación básica ground / non-ground usando grilla XY
    """

    ix = np.floor(x / cell_size).astype(int)
    iy = np.floor(y / cell_size).astype(int)

    grid = {}
    for i in range(len(z)):
        key = (ix[i], iy[i])
        grid.setdefault(key, []).append(z[i])

    ground_z = {
        k: np.percentile(v, ground_percentile)
        for k, v in grid.items()
    }

    is_ground = np.zeros(len(z), dtype=bool)
    for i in range(len(z)):
        if z[i] <= ground_z[(ix[i], iy[i])] + z_threshold:
            is_ground[i] = True

    return is_ground

## 5. Función: Visualización Rápida de Nube de Puntos

Función auxiliar para visualizar la nube de puntos en 2D con color según elevación.
Limita el número de puntos mostrados para evitar sobrecarga de memoria.

In [ ]:
def plot_lidar_fast(x, y, z, title="", max_points=200_000):
    if len(x) > max_points:
        idx = np.random.choice(len(x), max_points, replace=False)
        x, y, z = x[idx], y[idx], z[idx]

    plt.figure(figsize=(8, 6))
    sc = plt.scatter(x, y, c=z, s=1, cmap="terrain")
    plt.colorbar(sc, label="Z (m)")
    plt.title(title)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

## 6. Procesamiento Principal: Limpieza y Clasificación Básica

Itera sobre cada archivo LAS:
1. Lee la nube de puntos
2. Aplica limpieza de outliers
3. Clasifica puntos en ground/non-ground
4. Almacena resultados y estadísticas

In [ ]:
results = []
summary = []

for las_path in las_files:
    print("\nProcesando:", os.path.basename(las_path))

    las = laspy.read(las_path)

    x = np.asarray(las.x)
    y = np.asarray(las.y)
    z = np.asarray(las.z)

    # --- Limpieza ---
    mask_clean, stats = clean_lidar_points(x, y, z)

    # --- Suelo / no suelo ---
    ground_mask = classify_ground(
        x[mask_clean],
        y[mask_clean],
        z[mask_clean]
    )

    results.append({
        "file": os.path.basename(las_path),
        "x": x[mask_clean],
        "y": y[mask_clean],
        "z": z[mask_clean],
        "ground": ground_mask
    })

    stats["file"] = os.path.basename(las_path)
    stats["ground_points"] = int(ground_mask.sum())
    stats["non_ground_points"] = int((~ground_mask).sum())

    summary.append(stats)

    print("  Limpios:", stats["clean_points"])
    print("  Ground:", stats["ground_points"])
    print("  Non-ground:", stats["non_ground_points"])

## 7. Visualizar Nubes Limpias

Muestra cada archivo procesado como nube de puntos coloreada por elevación.

In [ ]:
for res in results:
    plot_lidar_fast(
        res["x"],
        res["y"],
        res["z"],
        title=f"Nube limpia – {res['file']}"
    )

## 8. Resumen de Procesamiento

Tabla con estadísticas de cada archivo procesado:
- Puntos totales, limpios y removidos
- Puntos clasificados como ground y non-ground

In [ ]:
df_summary = pd.DataFrame(summary)
df_summary

## 9. Función: Normalización de Alturas

Calcula la altura normalizada respecto al terreno local:
- `h_norm = z - z_terreno_local`
- Usa los 8 vecinos más cercanos del suelo (ground) para estimar el terreno local
- Utiliza árbol espacial (cKDTree) para búsqueda rápida de vecinos

In [ ]:
def normalize_height(x, y, z, ground_mask, k=8):
    """
    Calcula altura normalizada respecto al terreno local.
    h = z - z_terreno_local

    - Usa vecinos más cercanos SOLO del suelo
    """
    # Puntos de suelo
    ground_xy = np.column_stack((x[ground_mask], y[ground_mask]))
    ground_z = z[ground_mask]

    # Árbol espacial
    tree = cKDTree(ground_xy)

    # Consulta para todos los puntos
    pts = np.column_stack((x, y))
    _, idx = tree.query(pts, k=k)

    # Terreno local (mediana para robustez)
    z_ground_local = np.median(ground_z[idx], axis=1)

    h_norm = z - z_ground_local
    return h_norm

## 10. Calcular Alturas Normalizadas

Aplica la normalización de alturas a cada archivo procesado.

In [ ]:
for res in results:
    res["h_norm"] = normalize_height(
        res["x"],
        res["y"],
        res["z"],
        res["ground"]
    )

## 11. Función: Clasificación de Coberturas

Clasifica puntos en 4 categorías según altura normalizada y rugosidad:
- **ground**: Puntos de suelo
- **vegetation_low**: Vegetación baja (0.3 - 2.0 m)
- **vegetation_high**: Árboles y arbustos altos (>2.0 m, rugosos)
- **building**: Edificios (>2.0 m, suave/plano)

La rugosidad se calcula como la desviación estándar de alturas en un radio de 1.5 m.

In [ ]:
def classify_cover(x, y, h_norm, ground_mask, radius=1.5):
    """
    Clasifica coberturas:
    - ground
    - vegetation_low
    - vegetation_high
    - building
    """

    labels = np.full(len(h_norm), "unclassified", dtype=object)

    # Suelo
    labels[ground_mask] = "ground"

    # Vegetación baja
    veg_low = (h_norm > 0.3) & (h_norm <= 2.0)
    labels[veg_low] = "vegetation_low"

    # Candidatos altos
    high = h_norm > 2.0

    # Rugosidad local (std de h_norm)
    pts = np.column_stack((x, y))
    tree = cKDTree(pts)
    neighbors = tree.query_ball_point(pts, r=radius)

    roughness = np.zeros(len(h_norm))
    for i, idx in enumerate(neighbors):
        if len(idx) > 5:
            roughness[i] = np.std(h_norm[idx])

    # Edificaciones: alto + plano
    buildings = high & (roughness < 0.5)
    labels[buildings] = "building"

    # Vegetación alta: alto + rugoso
    vegetation_high = high & ~buildings
    labels[vegetation_high] = "vegetation_high"

    return labels

## 12. Clasificar Coberturas

Aplica la clasificación de coberturas a todos los archivos.

In [ ]:
for res in results:
    res["cover"] = classify_cover(
        res["x"],
        res["y"],
        res["h_norm"],
        res["ground"]
    )

## 13. Función: Visualizar Coberturas en 2D

Dibuja los puntos clasificados en planta XY con colores por tipo de cobertura.

In [ ]:
def plot_cover_xy(res):
    plt.figure(figsize=(8, 6))

    colors = {
        "ground": "saddlebrown",
        "vegetation_low": "limegreen",
        "vegetation_high": "darkgreen",
        "building": "gray"
    }

    for cls, col in colors.items():
        mask = res["cover"] == cls
        plt.scatter(
            res["x"][mask],
            res["y"][mask],
            s=1,
            c=col,
            label=cls
        )

    plt.title(f"Segregación de coberturas – {res['file']}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend(markerscale=5)
    plt.tight_layout()
    plt.show()

## 14. Visualizar Todas las Coberturas

Muestra el mapa de coberturas para cada archivo procesado.

In [ ]:
for res in results:
    plot_cover_xy(res)

## 15. Función: Interpolación de Grilla

Convierte puntos dispersos en una grilla regular usando interpolación.
Útil para crear modelos digitales continuos (DTM, DSM).

Parámetros:
- `resolution`: Tamaño de pixel (default: 1.0 m)
- `method`: Método de interpolación - "linear" o "nearest"

In [ ]:
def interpolate_grid(x, y, z, resolution=1.0, method="linear"):
    xi = np.arange(x.min(), x.max(), resolution)
    yi = np.arange(y.min(), y.max(), resolution)

    grid_x, grid_y = np.meshgrid(xi, yi)

    grid_z = griddata(
        (x, y),
        z,
        (grid_x, grid_y),
        method=method
    )

    return grid_x, grid_y, grid_z

## 16. Generar DTM (Modelo Digital del Terreno)

El DTM es un modelo que representa solo la elevación del terreno/suelo.
Se interpola usando solo los puntos clasificados como ground.
Resultado: grillas de X, Y, Z del DTM para cada archivo.

In [ ]:
for res in results:
    mask_ground = res["cover"] == "ground"

    res["DTM"] = interpolate_grid(
        res["x"][mask_ground],
        res["y"][mask_ground],
        res["z"][mask_ground],
        resolution=1.0,
        method="linear"
    )

## 17. Función: Generar DSM (Modelo Digital de Superficie)

El DSM representa la superficie visible (edificios, árboles, terreno).
Usa todos los puntos (no solo ground).
Utiliza interpolación "nearest" para evitar agujeros en zonas densas de vegetación.

In [ ]:
def interpolate_dsm(x, y, z, resolution=1.0):
    xi = np.arange(x.min(), x.max(), resolution)
    yi = np.arange(y.min(), y.max(), resolution)

    grid_x, grid_y = np.meshgrid(xi, yi)

    grid_z = griddata(
        (x, y),
        z,
        (grid_x, grid_y),
        method="nearest"  # evita agujeros
    )

    return grid_x, grid_y, grid_z

## 18. Generar DSM

Crea el modelo digital de superficie para cada archivo.

In [ ]:
for res in results:
    res["DSM"] = interpolate_dsm(
        res["x"],
        res["y"],
        res["z"],
        resolution=1.0
    )

## 19. Calcular nDSM (Modelo Digital de Normalizado)

El nDSM (Normalized Digital Surface Model) es la diferencia entre DSM y DTM.
Representa la altura de objetos sobre el terreno:
- nDSM = DSM - DTM
- Útil para identificar edificios y vegetación

In [ ]:
for res in results:
    _, _, dtm = res["DTM"]
    _, _, dsm = res["DSM"]

    res["nDSM"] = dsm - dtm

## 20. Función: Segmentación de Edificios

Agrupa puntos clasificados como edificios en clusters espaciales usando DBSCAN.
Cada cluster representa un edificio independiente.

Parámetros:
- `eps`: Distancia máxima entre vecinos (default: 2.0 m)
- `min_samples`: Puntos mínimos para formar un cluster (default: 50)

In [ ]:
def segment_buildings(x, y, h_norm, cover,
                      eps=2.0,
                      min_samples=50):
    """
    Segmenta edificios usando clustering espacial
    """
    mask_building = cover == "building"

    coords = np.column_stack((
        x[mask_building],
        y[mask_building]
    ))

    if len(coords) == 0:
        return None, None

    clustering = DBSCAN(
        eps=eps,
        min_samples=min_samples
    ).fit(coords)

    labels = clustering.labels_

    return mask_building, labels

## 21. Segmentar Edificios

Aplica clustering DBSCAN a los puntos de edificios de cada archivo.
Cada cluster recibe una etiqueta única de identificación.

In [ ]:
for res in results:
    res["building_mask"], res["building_labels"] = segment_buildings(
        res["x"],
        res["y"],
        res["h_norm"],
        res["cover"]
    )

## 22. Función: Cálculo de Métricas de Edificios

Para cada edificio segmentado, calcula:
- **area_m2**: Área del edificio (número de puntos × resolución²)
- **height_mean**: Altura media del edificio
- **height_max**: Altura máxima del edificio

In [ ]:
def building_metrics(res, resolution=1.0):
    metrics = []

    if res["building_labels"] is None:
        return metrics

    labels = res["building_labels"]
    unique_labels = set(labels)
    unique_labels.discard(-1)  # ruido

    bx = res["x"][res["building_mask"]]
    by = res["y"][res["building_mask"]]
    bh = res["h_norm"][res["building_mask"]]

    for lbl in unique_labels:
        mask = labels == lbl

        area = len(bx[mask]) * (resolution ** 2)
        height_mean = bh[mask].mean()
        height_max = bh[mask].max()

        metrics.append({
            "file": res["file"],
            "building_id": int(lbl),
            "area_m2": area,
            "height_mean": height_mean,
            "height_max": height_max
        })

    return metrics

## 23. Calcular Métricas para Todos los Edificios

Recopila las métricas de edificios de todos los archivos procesados.

In [ ]:
building_table = []

for res in results:
    building_table.extend(building_metrics(res))

## 24. Tabla de Métricas de Edificios

Muestra un resumen de todos los edificios detectados con sus características.

In [ ]:
df_buildings = pd.DataFrame(building_table)
df_buildings

## 25. Resumen Final de Coberturas

Tabla con el conteo total de puntos por tipo de cobertura para cada archivo.

In [ ]:
final_summary = []

for res in results:
    final_summary.append({
        "file": res["file"],
        "total_points": len(res["x"]),
        "ground_points": int((res["cover"] == "ground").sum()),
        "vegetation_points": int(
            ((res["cover"] == "vegetation_low") |
             (res["cover"] == "vegetation_high")).sum()
        ),
        "building_points": int((res["cover"] == "building").sum())
    })

df_summary_final = pd.DataFrame(final_summary)
df_summary_final

## 26. Función: Exportar a GeoTIFF

Convierte las grillas interpoladas (DTM, DSM, nDSM) a archivos GeoTIFF georreferenciados.
GeoTIFF es un formato estándar para datos geoespaciales raster.

In [ ]:
def export_geotiff(grid_x, grid_y, grid_z, filename):
    pixel_size = grid_x[0,1] - grid_x[0,0]

    transform = from_origin(
        grid_x.min(),
        grid_y.max(),
        pixel_size,
        pixel_size
    )

    with rasterio.open(
        filename,
        "w",
        driver="GTiff",
        height=grid_z.shape[0],
        width=grid_z.shape[1],
        count=1,
        dtype=grid_z.dtype,
        crs=None,
        transform=transform
    ) as dst:
        dst.write(grid_z, 1)

## 27. Exportar Modelos a GeoTIFF

Guarda DTM, DSM y nDSM de cada archivo como archivos GeoTIFF en la carpeta `outputs`.
Estos archivos pueden abrirse en software SIG (QGIS, ArcGIS).

In [ ]:
os.makedirs("outputs", exist_ok=True)

for res in results:
    name = res["file"].replace(".las", "")

    export_geotiff(*res["DTM"], f"outputs/{name}_DTM.tif")
    export_geotiff(*res["DSM"], f"outputs/{name}_DSM.tif")
    export_geotiff(res["DSM"][0], res["DSM"][1], res["nDSM"],
                   f"outputs/{name}_nDSM.tif")

## 28. Exportar Tablas de Resumen

Guarda en CSV:
- `summary_lidar.csv`: Conteo de puntos por cobertura
- `buildings_metrics.csv`: Métricas de edificios detectados

In [ ]:
df_summary_final.to_csv("outputs/summary_lidar.csv", index=False)
df_buildings.to_csv("outputs/buildings_metrics.csv", index=False)